In [1]:
%load_ext autoreload
%autoreload 2
# System imports
import copy
import numpy as np
import time
import itertools
import os
import tempfile
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp

# Main C3 objects
from c3.c3objs import Quantity as Qty
from c3.parametermap import ParameterMap as PMap
from c3.experiment import Experiment as Exp
from c3.system.model import Model as Mdl
from c3.generator.generator import Generator as Gnr

# Building blocks
import c3.generator.devices as devices
import c3.signal.gates as gates
import c3.system.chip as chip
import c3.signal.pulse as pulse
import c3.system.tasks as tasks

# Libs and helpers
import c3.libraries.algorithms as algorithms
import c3.libraries.hamiltonians as hamiltonians
import c3.libraries.fidelities as fidelities
import c3.libraries.envelopes as envelopes
import c3.utils.qt_utils as qt_utils
import c3.utils.tf_utils as tf_utils
import examples.two_qubit_chip_exp
import examples.single_qubit_rabi12
import useful
from scipy.optimize import minimize_scalar
from c3.optimizers.c1 import C1
import c3.libraries.algorithms as algorithms

#%matplotlib inline
#import mpld3
#mpld3.enable_notebook()

import matplotlib
import time

#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution
#tf.config.run_functions_eagerly(True)
print(tf.executing_eagerly())

True


In [ ]:
exp = examples.single_qubit_rabi12.create_experiment()
exp2 = examples.single_qubit_rabi12.create_experiment(gatetime=20e-9, anhar=-1e7)

In [3]:
psi_init = [[0] * 3]
psi_init[0][0] = 1

init_state = tf.transpose(tf.constant(psi_init, tf.complex128))
print(init_state)
pi01_seq = ['PI01p']
rx90_seq = ['RX90p']
rx_seq = ['RXp']
rabi12_seq = ['RXp','RXp12','Id']

tf.Tensor(
[[1.+0.j]
 [0.+0.j]
 [0.+0.j]], shape=(3, 1), dtype=complex128)


In [4]:
#TODO change amplitude gauss here in loop

amps = np.arange(0.1,1,1)

In [6]:
for amp in amps:
    exp.change_pi_amp(amp)
    exp.get_gates()
    t,pop = useful.plot_dynamics(exp, init_state, pi01_seq)
    print(pop[-1])

KeyError: 'PI01p'

In [ ]:
exp.get_gates()
_,_ = useful.plot_dynamics(exp, init_state, rabi12_seq)


In [ ]:
exp.pmap.instructions['Id'].t_end

In [ ]:
exp.evaluate([pi01_seq],init_state)

Optimize RXp

In [ ]:
opt_gates = ["RXp"]
gateset_opt_map=[
    [
      ("RXp", "d1", "gauss", "amp"),
    ],
    [
      ("RXp", "d1", "gauss", "freq_offset"),
    ],
    [
      ("RXp", "d1", "gauss", "xy_angle"),
    ],
    [
      ("RXp", "d1", "gauss", "delta"),
    ]
]
log_dir = os.path.join(tempfile.TemporaryDirectory().name, "c3logs")

opt = C1(
    dir_path=log_dir,
    fid_func=fidelities.average_infid_set,
    fid_subspace=["Q1"],
    pmap=exp.pmap,
    algorithm=algorithms.lbfgs,
    options={"maxfun" : 10},
    run_name="better_RX"
)
exp.pmap.set_opt_map(gateset_opt_map)
exp.set_opt_gates(opt_gates)
opt.set_exp(exp)

opt.optimize_controls()
print(opt.current_best_params)
print(opt.current_best_goal)


After opt

In [ ]:
_,_ = useful.plot_dynamics(exp, init_state, rabi12_seq)

Rabi 12


In [ ]:
amps12 = np.arange(0.1,2.1,0.1)
pops = np.zeros((len(amps12),3))

In [ ]:
exp.set_opt_gates(['RXp','RXp12','Id'])
for count, amp in enumerate(amps12):
    exp.change_pi12_amp(amp)
    exp.get_gates()
    pops[count] = np.array(exp.evaluate([rabi12_seq],init_state))[0].T




In [ ]:
useful.plot_rabi12(pops,amps12)
